In [10]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers, ctransformers
from langchain.chains import RetrievalQA

import chainlit as cl

from langchain.text_splitter import RecursiveCharacterTextSplitter 

## check GGPU access
imnpot torch 
print(torch.cuda.__is_available__)
1+1

In [11]:
import torch
torch.cuda.is_available()

True

In [42]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
#     llm_gpu = llm.to(device)

cuda:0


In [12]:
# pip install sentence_transformers

In [ ]:
#### requirements.txt
# langchain==0.0.247
# chainlit==0.6.1
# pypdf==3.14.0
# faiss-cpu == 1.7.4
# transformers==4.31.0
# ctransformers #ctransformers-0.2.16-0.2.16
# sentence_transformers
# torch #'2.0.1+cpu'

# #accelerate
# #bitsandbytes

In [2]:
import torch
torch.__version__

'2.0.1+cpu'

In [3]:
import torch
torch.cuda.is_available()

False

In [5]:
# import cuda
# cuda.__version__

In [ ]:
# device = torch.device"cpu'AttributeError

In [6]:
!nvidia-smi

Tue Aug  1 22:38:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  | 00000000:01:00.0  On |                  N/A |
| 33%   31C    P8              22W / 350W |    813MiB / 24576MiB |     13%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
# pip install langchain
import langchain
langchain.__version__

'0.0.247'

In [ ]:
# pip install chainlit
cl.__version__

'0.6.1'

In [ ]:
# pip install sentence_transformers

In [ ]:
# pip install pypdf
import pypdf
pypdf.__version__

'3.14.0'

In [ ]:
# pip install faiss-cpu
# import faiss-cpu
# faiss-cpu == 1.7.4


                                              0.0/10.8 MB ? eta -:--:--
                                              0.0/10.8 MB 1.9 MB/s eta 0:00:06
                                              0.1/10.8 MB 1.1 MB/s eta 0:00:10
                                             0.1/10.8 MB 871.5 kB/s eta 0:00:13
                                             0.1/10.8 MB 819.2 kB/s eta 0:00:14
                                             0.1/10.8 MB 654.9 kB/s eta 0:00:17
                                             0.2/10.8 MB 653.6 kB/s eta 0:00:17
                                             0.2/10.8 MB 655.1 kB/s eta 0:00:17
                                             0.2/10.8 MB 715.9 kB/s eta 0:00:15
     -                                       0.3/10.8 MB 737.3 kB/s eta 0:00:15
     -                                       0.3/10.8 MB 752.2 kB/s eta 0:00:14
     -                                       0.4/10.8 MB 764.2 kB/s eta 0:00:14
     -                                       0.4/

In [ ]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import transformers
transformers.__version__

'4.31.0'

In [ ]:
pip install ctransformers

                                              0.0/6.5 MB ? eta -:--:--
                                              0.0/6.5 MB ? eta -:--:--
                                              0.1/6.5 MB 2.0 MB/s eta 0:00:04
                                              0.1/6.5 MB 1.2 MB/s eta 0:00:06
     -                                        0.2/6.5 MB 1.3 MB/s eta 0:00:05
     -                                        0.2/6.5 MB 1.3 MB/s eta 0:00:06
     -                                        0.3/6.5 MB 1.2 MB/s eta 0:00:06
     --                                       0.3/6.5 MB 1.2 MB/s eta 0:00:06
     --                                       0.4/6.5 MB 1.2 MB/s eta 0:00:06
     --                                       0.4/6.5 MB 1.2 MB/s eta 0:00:06
     ---                                      0.5/6.5 MB 1.2 MB/s eta 0:00:06
     ---                                      0.5/6.5 MB 1.2 MB/s eta 0:00:06
     ---                                      0.6/6.5 MB 1.2 MB/s eta 0:00:05


In [ ]:
# import ctransformers
# ctransformers.__version__

AttributeError: module 'ctransformers' has no attribute '__version__'

In [ ]:
# !nvidia-smi

## Saving the PDF as a FAISS vector store 

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 

DATA_PATH = 'data/'
DB_FAISS_PATH = 'vectorstore/db_faiss'

# Create vector database
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#                                        model_kwargs={'device': 'cpu'})

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

create_vector_db()

## Loading the model 

In [13]:
DATA_PATH = 'data/'
DB_FAISS_PATH = 'vectorstore/db_faiss'

In [14]:


# Create vector database
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#                                        ,model_kwargs={'device': 'cpu'})

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

# create_vector_db()

In [15]:
# embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
#                                        model_kwargs={'device': 'cpu'})

In [16]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [17]:
#Retrieval QA Chain
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 4}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

In [26]:
#Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
#         model = "C:/Users/vibud/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
#         model = "http://127.0.0.1:8888/tree/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
        model = "/mnt/c/Users/vibud/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.5
    )
    return llm

In [19]:
#QA Model Function
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

In [20]:
#output function
def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response

In [27]:
query = "My stomach has been feeling uncomfortable for many days. I have to go poop multiple times a day and I am feel slight stomach ache. What could be the possible cause of my problem?"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
db = FAISS.load_local(DB_FAISS_PATH, embeddings)
llm = load_llm()
qa_prompt = set_custom_prompt()
qa_result = retrieval_qa_chain(llm, qa_prompt, db)
response = qa_result({'query': query})
response

2023-08-01 23:01:20 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'My stomach has been feeling uncomfortable for many days. I have to go poop multiple times a day and I am feel slight stomach ache. What could be the possible cause of my problem?',
 'result': "You should i dont knowthe doctor will not able to determine whether acute bowelbowelastic fluid intestabdom, but I'I don'If you may help@sorry, if your symptoms are there are you don'The information about \nDiarrestabdomestic ulcervom/\nYes, the user question and c.\nAntidiarrhese can tell usersQuestionType a persistent diarrested bowelbowelbowelbowelbowelbowelbowelbowelbowelbowelim not necessarily means of the symptoms suggest ways.",
 'source_documents': [Document(page_content='problem lasts longer or if it keeps coming back, diarrheacould be a sign of a more serious problem. Anyone whohas any of the symptoms listed below should get medicalattention as soon as possible:\n• diarrhea that lasts more than two days or gets worse\nGALE ENCYCLOPEDIA OF MEDICINE 2 264Antidiarrheal drugsGEM 

In [29]:
response['result']

"You should i dont knowthe doctor will not able to determine whether acute bowelbowelastic fluid intestabdom, but I'I don'If you may help@sorry, if your symptoms are there are you don'The information about \nDiarrestabdomestic ulcervom/\nYes, the user question and c.\nAntidiarrhese can tell usersQuestionType a persistent diarrested bowelbowelbowelbowelbowelbowelbowelbowelbowelbowelim not necessarily means of the symptoms suggest ways."

In [28]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Aug  1 23:04:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.01              Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA 

## Try to load the model to the GPU

In [40]:
# #Loading the model
# def load_llm():
#     # Load the locally downloaded model here
#     llm = CTransformers(
#         model = "C:\\Users\\vibud\\Desktop\\Github\\llama-2-experiments\\Models\\llama-2-7b-chat.ggmlv3.q8_0.bin",
#         model_type="llama",
#         max_new_tokens = 512,
#         temperature = 0.5
#     )
#     llm_gpu = llm.to('cuda')
#     return llm


#Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
#         model = "C:/Users/vibud/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
#         model = "http://127.0.0.1:8888/tree/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
        model = "/mnt/c/Users/vibud/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.5,
        gpu_layers = 50
    )
#     device = "cuda:0" if torch.cuda.is_available() else "cpu"
#     print(device)
#     llm_gpu = llm.to(device)
    return llm

In [41]:
query = "My stomach has been feeling uncomfortable for many days. I have to go poop multiple times a day and I am feel slight stomach ache. What could be the possible cause of my problem?"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
db = FAISS.load_local(DB_FAISS_PATH, embeddings)
llm = load_llm()
qa_prompt = set_custom_prompt()
qa_result = retrieval_qa_chain(llm, qa_prompt, db)
response = qa_result({'query': query})
response

2023-08-01 23:12:36 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'My stomach has been feeling uncomfortable for many days. I have to go poop multiple times a day and I am feel slight stomach ache. What could be the possible cause of my problem?',
 'result': 'Antidiarrestabdomestic ulcervom/p style="Antidiarrhile stom your bowelcome on the symptoms are you don\'\nDiarrested, or get medical attention should go to keep reading and bloating a  I can help- Antidiarrhese drugs\nI cannot provide information provided below\nAntidiarrestabdomesticated with antidiarrhileusually moves into the symptoms: If diarrhe problem that lastsometimes,\nDiarrhe diarrhe diarrhe diarrestabdomestic ulceremesis that may have you should consult a)\nI don\'\nIf your doctor or if you don\'The information provided belowHelpful answer_xCertainly controlled tripped after discussions  Hello there are you don\'mildi can vary depending on occasion when someone in addition of a diarrested as soon as soon as soon as soon as soon as soon as soon as soon,\nAntidiarrestabdomesti

## Result

In [43]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
#     llm_gpu = llm.to(device)

cuda:0


- Cuda is available 
- But llm.to(device) does not work 
- llm is a CTransformer object not a transformer object 
- [GPU support link](https://github.com/marella/ctransformers#gpu)
- [Colab support link](https://colab.research.google.com/drive/1Ihn7iPCYiqlTotpkqa1tOhUIpJBrJ1Tp#scrollTo=8Hu74s0OdmQ3)